# Neural Network Training

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from utils.Block import Model
from utils.train import train, to_tensor
from utils.metrics import modified_smape

gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(
    gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*5)]
)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

X_submit = pd.concat([ 
    pd.read_csv('./data/300_X_submit.csv'), 
    pd.read_csv('./data/400_X_submit_preds.csv', header=None)
], axis=1)

X_train = pd.concat([
    pd.read_csv('./data/300_X_train.csv'), 
    pd.read_csv('./data/400_X_train_preds.csv', header=None)
], axis=1)

X_test = pd.concat([
    pd.read_csv('./data/300_X_test.csv'), 
    pd.read_csv('./data/400_X_test_preds.csv', header=None)
], axis=1)

X_submit = X_submit.set_index('id')
X_train  = X_train.set_index('id')
X_test   = X_test.set_index('id')

y_submit = pd.read_csv('./data/004_test.csv', index_col='id')
y_train = pd.read_csv('./data/400_y_train.csv', index_col='id')
y_test = pd.read_csv('./data/400_y_test.csv', index_col='id')

X = X_train.values
y = y_train.values

X.shape

## Train-validation split

In [ ]:
model = Model(hiddens=[176,128])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.11111, random_state=42)
X_train, y_train, X_valid, y_valid = to_tensor(X_train, y_train, X_valid, y_valid)

In [ ]:
model = train(model, X_train, y_train, X_valid, y_valid, batch_size=1024, lr=0.001)

## Save model

In [ ]:
model_name = '061_stack_nn_176.128'
model.save_weights('M_336/checkpoints/'+model_name+'/model')

# Inferencing
## Load model

In [ ]:
model_name = '610_stack_nn_176.128'
latest = tf.train.latest_checkpoint('M_336/checkpoints/'+model_name)
model.load_weights(latest)
modified_smape(np.squeeze(y_test.values), np.squeeze(np.array(model(X_test.values))))

## Predict test & submit

In [ ]:
X_test_pred   = np.squeeze(np.array(model(X_test.values)))
X_submit_pred = np.squeeze(np.array(model(X_submit.values)))

X_test_pred.shape

In [ ]:
np.savetxt('data/061_X_test_stack_nn_176.128_pred.csv'    , X_test_pred  , delimiter=",")
np.savetxt('data/061_X_submit_stack_nn_176.128_pred.csv'  , X_submit_pred, delimiter=",")